In [106]:
# Originally by Tom Roeschinger (2020)
import pandas as pd
import numpy as np
import requests
from datetime import datetime as dt
from io import StringIO
import os
import us

In [195]:
def clean_df(df, val_name="Cases"):
    """Cleans up dataframe"""
    
    # convert to longform
    df = df.melt(value_vars=df.columns[4:],id_vars=df.columns[0:4], var_name="Date", value_name=val_name)
    
    # clean datatypes
    df['Province/State'] = df['Province/State'].astype('string')
    df['Country/Region'] = df['Country/Region'].astype('string')
    df['Date'] = df['Date'].astype('datetime64')
    return df

def tidy_provinces(df):
    """Split complex state/province names (US only so far)"""
    # Clean so DC appears as a single 'state'
    df.loc[df['Province/State'].str.contains('Washington, D.C.'),
           'Province/State'] = 'Washington DC'
    df['City/County'] = pd.Series(np.repeat(pd.NA, len(df)), dtype='string')
    
    # Select Province/State combinations with a comma
    comma_filter = df['Province/State'].str.contains(',')
    # Extract city or county name
    cities_counties = (df[comma_filter]['Province/State']
                       .apply(lambda string: string.split(',')[0].strip())
                       .astype('string'))
    # Extract state abbreviation and fix as fullname
    states = (df[comma_filter]['Province/State']
              .apply(lambda string: string.split(',')[1].strip())
              .apply(lambda state: us.states.lookup(state).name)
              .astype('string'))
    # Assign into data frame
    df.loc[comma_filter, 'City/County'] = cities_counties
    df.loc[comma_filter, 'Province/State'] = states
    
    return df[['City/County'] +  df.columns[df.columns != 'City/County'].to_list()]

In [196]:
# urls for data in Johns Hopkins github repository
urls = {'Confirmed' : "https://raw.github.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv",
        'Deaths' : "https://raw.github.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv",
        'Recovered' : "https://raw.github.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv"}

output_dfs = {}

for condition, url in urls.items():
    request = requests.get(url)
    txt = StringIO(req.text)
    df = pd.read_csv(txt)
    output_dfs[condition] = clean_df(df,val_name=condition)
    
df = (output_dfs['Confirmed']
      .merge(output_dfs['Deaths'])
      .merge(output_dfs['Recovered']))

df = tidy_provinces(df)

209          Kitsap
210          Solano
211      Santa Cruz
212            Napa
213         Ventura
            ...    
29615       Socorro
29616    Bernalillo
29617       Oakland
29618         Wayne
29619    New Castle
Name: Province/State, Length: 11590, dtype: string


In [203]:
df[df['Country/Region']=='US']

,City/County,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
98,<NA>,Washington,US,47.4009,-121.4905,2020-01-22,0,0,0
99,<NA>,New York,US,42.1657,-74.9481,2020-01-22,0,0,0
100,<NA>,California,US,36.1162,-119.6816,2020-01-22,0,0,0
101,<NA>,Massachusetts,US,42.2302,-71.5301,2020-01-22,0,0,0
102,<NA>,Diamond Princess,US,35.4437,139.6380,2020-01-22,0,0,0
...,...,...,...,...,...,...,...,...,...
29619,New Castle,Delaware,US,39.5393,-75.6674,2020-03-22,0,0,0
29634,<NA>,Alabama,US,32.3182,-86.9023,2020-03-22,0,0,0
29637,<NA>,Puerto Rico,US,18.2208,-66.5901,2020-03-22,0,0,0
29651,<NA>,Guam,US,13.4443,144.7937,2020-03-22,0,0,0
